In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from dynamicslib.common import *
from dynamicslib.consts import *
from dynamicslib.continuation import *
from dynamicslib.targeter import *
from dynamicslib.plotting import *

In [ ]:
# X = [X, Z, vy, tf/2]
# f = [y, vx, vz]


def xtf2X(x0, tf):
    return np.array([x0[0], x0[2], x0[-2], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, X[1], 0, X[2], 0]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [
            [stm[1, 0], stm[1, 2], stm[1, -2], eomf[1]],
            [stm[-3, 0], stm[-3, 2], stm[-3, -2], eomf[-3]],
            [stm[-1, 0], stm[-1, 2], stm[-1, -2], eomf[-1]],
        ]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3], xf[-1]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-12)

In [ ]:
# get the linear IC


def get_IC(Lpt, mu=muEM):
    assert Lpt in [1, 2, 3]
    xL = get_Lpts()[0, Lpt - 1]
    step = 1e-2
    eq_soln = np.array([xL, 0, 0, 0, 0])
    A = get_A(eq_soln, mu)
    s = np.sqrt(np.abs(A[-1, 2]))

    x0 = np.array([xL, 0, step, 0, -step/10, 0])
    period = 2 * np.pi / s
    X0 = xtf2X(x0, period)
    return X0

In [ ]:
Xs1 = []
e1 = []
Xs1, e1 = arclen_cont(get_IC(1), func, [0, 1, 0, 0], 5e-3, 4.5, 1e-10)

In [ ]:
Xs2 = []
e2 = []
Xs2, e2 = arclen_cont(get_IC(2), func, [0, 1, 0, 0], 5e-3, 4, 1e-10)

In [ ]:
Xs3 = []
e3 = []
Xs3, e3 = arclen_cont(get_IC(3), func, [0, 1, 0, 0], 5e-3, 3.5, 1e-10)

In [ ]:
es = [e1, e2, e3]
XsAll = [Xs1[::], Xs2[::], Xs3[::]]

param_names = [
    "Index",
    "Initial x",
    "Initial z",
    "Initial vy",
    "Jacobi Constant",
    "Period",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
]

xyzs = []
data = []


for Lp in range(3):
    xyzs_lp = []
    data_lp = []
    iterator = list(enumerate(zip(XsAll[Lp], es[Lp])))
    for i, (X, evals) in tqdm(iterator):
        Xcp = X.copy()
        xyzs_lp.append(prop_ic(Xcp, X2xtf))
        args = np.argsort(np.abs(evals))
        eigs = evals[[*args[:2], *args[-2:]]]
        stabind = np.max([np.abs(lam) + 1 / np.abs(lam) / 2 for lam in evals])
        data_lp.append([i, *Xcp[:-1], *get_JC_tf(Xcp, X2xtf), stabind, *eigs])
    xyzs.append(xyzs_lp)
    data.append(data_lp)